In [6]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import requests
import pandas as pd

In [23]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path":'chromedriver.exe'}
    return Browser("chrome", **executable_path, headless=False)


In [45]:
def scrape_info():
    browser = init_browser()
    
    # Visit visitcostarica.herokuapp.com
    news_url = "https://mars.nasa.gov/news/"
    browser.visit(news_url)

    time.sleep(1)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")

    # Scrape news title
    news_title = soup.find_all("div", class_="content_title")[1].text
    news_p = soup.find("div", class_="article_teaser_body").text
    print(news_title)
    print(news_p)

In [46]:
scrape_info()

The Detective Aboard NASA's Perseverance Rover
An instrument called SHERLOC will, with the help of its partner WATSON, hunt for signs of ancient life by detecting organic molecules and minerals.


# Mars Image

In [57]:
def scrape_featured_image():
    browser = init_browser()

    jpl_url="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(jpl_url)

    html = browser.html
    soup = bs(html, "html.parser")

    featured_image_url = 'http://www.jpl.nasa.gov' + soup.find("article").find('a')['data-fancybox-href']
    return featured_image_url

In [58]:
scrape_featured_image()

'http://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16192_ip.jpg'

# Mars Weather

In [54]:
import requests
# browser = init_browser()
def scrape_mars_weather():
    twitter_url="https://twitter.com/marswxreport?lang=en"
# browser.visit(twitter_url)
    data = requests.get(twitter_url)

    soup = bs(data.text, "html.parser")
    tweets = soup.find_all('div', class_="js-tweet-text-container")

    for tweet in tweets:
        tweet.find('a', class_="twitter-timeline-link u-hidden").decompose()
        if "sol " in tweet.text:
            mars_weather = tweet.text.strip()
            break

    print(mars_weather)

In [55]:
scrape_mars_weather()

InSight sol 534 (2020-05-27) low -92.8ºC (-135.1ºF) high 0.9ºC (33.7ºF)
winds from the SW at 4.8 m/s (10.8 mph) gusting to 18.8 m/s (42.1 mph)
pressure at 7.20 hPa


# Mars Facts

In [12]:
import pandas as pd


In [13]:
def scrape_mars_facts():
    fact_url="https://space-facts.com/mars/"
#Use Pandas to scrape the table containing facts about the planet
    fact_table = pd.read_html(fact_url)
    fact_df = fact_table[0]
    fact_df.columns=['Mars','Facts']
    fact_df.columns=['Mars','Facts']
    fact_df.set_index('Mars', inplace=True)
    html_table = fact_df.to_html()
    html_table = html_table.replace('\n', '')
    return html_table

In [14]:
scrape_mars_facts()

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Facts</th>    </tr>    <tr>      <th>Mars</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

# Mars Hemispheres

In [38]:
def scrape_mars_hemispheres():
#visit hemisphere website
    h_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
    response = requests.get(h_url)

#Parsel url with Beautiful Soup
    soup = bs(response.text, 'html.parser')

#Retreive all items that contain mars hemispheres information
    items = soup.find_all('div', class_='item')

#Create empty list for hemisphere urls
    h_image_urls =[]
    hemisphere_main_url = 'https://astrogeology.usgs.gov'

    #browser = init_browser()
    for i in items:
        title = i.find('h3').text
        partial_img_url = i.find('a', class_='itemLink product-item')['href']
        #browser.visit(hemisphere_main_url + partial_img_url)
        res = requests.get(hemisphere_main_url + partial_img_url)
        partial_img_html = res.text #browser.html
        soup = bs(partial_img_html, 'html.parser')
        img_url = hemisphere_main_url + soup.find('img', class_='wide-image')['src']
        h_image_urls.append({'title':title,'img_url':img_url})
        
    return h_image_urls

In [39]:
scrape_mars_hemispheres()

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]